In [ ]:
"""
=============================================================
 Financial News Scraper v2 - RSS-Based (ไม่ถูก block)
=============================================================
 ใช้ RSS Feed ทั้งหมด เพื่อหลีกเลี่ยงปัญหา 401/403
 
 แหล่ง RSS ที่ใช้:
   1. Yahoo Finance RSS        - finance.yahoo.com
   2. Investing.com RSS        - investing.com
   3. CNBC RSS                 - cnbc.com
   4. Financial Times RSS      - ft.com
   5. Seeking Alpha RSS        - seekingalpha.com
   6. Reuters RSS (Newsfeed)   - feeds.reuters.com
   7. AP Business RSS          - apnews.com
   8. WSJ RSS                  - wsj.com
   9. Barron's RSS             - barrons.com
  10. Forbes Finance RSS       - forbes.com

 Target: 500-1,000 รายการ
=============================================================
"""

import requests
from bs4 import BeautifulSoup
import feedparser          # pip install feedparser
import pandas as pd
import time
import random
import re
import json
import os
from datetime import datetime
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)
logger = logging.getLogger(__name__)

# ══════════════════════════════════════════════════════════
#  RSS FEED LIST - ทั้งหมดเป็น public feed ไม่ถูก block
# ══════════════════════════════════════════════════════════

RSS_FEEDS = [
    # ── Yahoo Finance ──────────────────────────────────────
    {"source": "Yahoo Finance", "url": "https://finance.yahoo.com/news/rssindex"},
    {"source": "Yahoo Finance - S&P500", "url": "https://finance.yahoo.com/rss/headline?s=^GSPC"},
    {"source": "Yahoo Finance - Dow",    "url": "https://finance.yahoo.com/rss/headline?s=^DJI"},
    {"source": "Yahoo Finance - NASDAQ", "url": "https://finance.yahoo.com/rss/headline?s=^IXIC"},
    {"source": "Yahoo Finance - Gold",   "url": "https://finance.yahoo.com/rss/headline?s=GC=F"},
    {"source": "Yahoo Finance - Oil",    "url": "https://finance.yahoo.com/rss/headline?s=CL=F"},

    # ── Investing.com ──────────────────────────────────────
    {"source": "Investing.com - Stock Market", "url": "https://www.investing.com/rss/news_285.rss"},
    {"source": "Investing.com - Economy",      "url": "https://www.investing.com/rss/news_1.rss"},
    {"source": "Investing.com - Forex",        "url": "https://www.investing.com/rss/news_25.rss"},
    {"source": "Investing.com - Commodities",  "url": "https://www.investing.com/rss/news_8.rss"},
    {"source": "Investing.com - Crypto",       "url": "https://www.investing.com/rss/news_301.rss"},

    # ── CNBC ───────────────────────────────────────────────
    {"source": "CNBC - Top News",     "url": "https://www.cnbc.com/id/100003114/device/rss/rss.html"},
    {"source": "CNBC - Finance",      "url": "https://www.cnbc.com/id/10000664/device/rss/rss.html"},
    {"source": "CNBC - Earnings",     "url": "https://www.cnbc.com/id/15839135/device/rss/rss.html"},
    {"source": "CNBC - Markets",      "url": "https://www.cnbc.com/id/20910258/device/rss/rss.html"},
    {"source": "CNBC - Economy",      "url": "https://www.cnbc.com/id/20910274/device/rss/rss.html"},
    {"source": "CNBC - World Markets","url": "https://www.cnbc.com/id/15839069/device/rss/rss.html"},

    # ── Reuters (Newsfeed public RSS) ──────────────────────
    {"source": "Reuters - Business", "url": "https://feeds.reuters.com/reuters/businessNews"},
    {"source": "Reuters - Markets",  "url": "https://feeds.reuters.com/reuters/companyNews"},
    {"source": "Reuters - US",       "url": "https://feeds.reuters.com/Reuters/domesticNews"},

    # ── AP News ────────────────────────────────────────────
    {"source": "AP - Business",  "url": "https://rsshub.app/apnews/topics/business"},
    {"source": "AP - Economy",   "url": "https://rsshub.app/apnews/topics/economy"},
    {"source": "AP - Finance",   "url": "https://rsshub.app/apnews/topics/financial-markets"},

    # ── Forbes ─────────────────────────────────────────────
    {"source": "Forbes - Business",   "url": "https://www.forbes.com/business/feed/"},
    {"source": "Forbes - Markets",    "url": "https://www.forbes.com/money/feed/"},
    {"source": "Forbes - Investing",  "url": "https://www.forbes.com/investing/feed/"},

    # ── MarketWatch (RSS ไม่ถูก block) ────────────────────
    {"source": "MarketWatch - Top Stories",  "url": "https://feeds.content.dowjones.io/public/rss/mw_topstories"},
    {"source": "MarketWatch - Market Pulse", "url": "https://feeds.content.dowjones.io/public/rss/mw_marketpulse"},
    {"source": "MarketWatch - Real Estate",  "url": "https://feeds.content.dowjones.io/public/rss/mw_realestate"},

    # ── Seeking Alpha ──────────────────────────────────────
    {"source": "Seeking Alpha - Market News", "url": "https://seekingalpha.com/market_currents.xml"},
    {"source": "Seeking Alpha - Wall Street", "url": "https://seekingalpha.com/feed/wall-street-breakfast"},

    # ── Barron's ───────────────────────────────────────────
    {"source": "Barron's", "url": "https://www.barrons.com/xml/rss/3_7510.xml"},

    # ── The Economist ──────────────────────────────────────
    {"source": "The Economist - Finance", "url": "https://www.economist.com/finance-and-economics/rss.xml"},
    {"source": "The Economist - Business","url": "https://www.economist.com/business/rss.xml"},

    # ── Motley Fool ────────────────────────────────────────
    {"source": "Motley Fool", "url": "https://www.fool.com/feeds/index.aspx"},

    # ── Nasdaq ─────────────────────────────────────────────
    {"source": "Nasdaq - Original", "url": "https://www.nasdaq.com/feed/rssoutbound?category=Original+Articles"},
    {"source": "Nasdaq - Markets",  "url": "https://www.nasdaq.com/feed/rssoutbound?category=Markets"},
]



2026-02-18 16:39:12,449 [INFO] 🚀 เริ่ม RSS Scraping เป้าหมาย 700 รายการ จาก 37 feeds
2026-02-18 16:39:12,455 [INFO]   [ 1/37] Yahoo Finance...
2026-02-18 16:40:59,216 [INFO]          +48 รายการ | รวม 48/700
2026-02-18 16:40:59,216 [INFO]   [ 2/37] Yahoo Finance - S&P500...
2026-02-18 16:41:42,390 [INFO]          +20 รายการ | รวม 68/700
2026-02-18 16:41:42,390 [INFO]   [ 3/37] Yahoo Finance - Dow...
2026-02-18 16:42:02,444 [INFO]          +10 รายการ | รวม 78/700
2026-02-18 16:42:02,445 [INFO]   [ 4/37] Yahoo Finance - NASDAQ...
2026-02-18 16:42:12,674 [INFO]          +5 รายการ | รวม 83/700
2026-02-18 16:42:12,674 [INFO]   [ 5/37] Yahoo Finance - Gold...
2026-02-18 16:43:38,174 [INFO]          +19 รายการ | รวม 102/700
2026-02-18 16:43:38,175 [INFO]   [ 6/37] Yahoo Finance - Oil...
2026-02-18 16:45:22,121 [INFO]          +18 รายการ | รวม 120/700
2026-02-18 16:45:22,122 [INFO]   [ 7/37] Investing.com - Stock Market...
2026-02-18 16:45:34,482 [INFO]          +1 รายการ | รวม 121/700
2026-02-


📊 TOKENIZATION COMPARISON
Sample: An Israeli Defense Forces reservist and a civilian have been indicted for allegedly using classified...

🔤 NLTK (Word-level)
   Tokens : 44
   Sample : ['israeli', 'defense', 'forces', 'reservist', 'civilian', 'indicted', 'allegedly', 'using']

🔤 spaCy (Word-level)
   Tokens : 43
   Sample : ['israeli', 'defense', 'forces', 'reservist', 'civilian', 'indict', 'allegedly', 'classified']

🔤 GPT-2 BPE (Subword)
   Tokens : 90
   Sample : ['An', 'ĠIsraeli', 'ĠDefense', 'ĠForces', 'Ġreserv', 'ist', 'Ġand', 'Ġa']

💡 spaCy ใช้ lemmatization (running→run) ทำให้ vocab เล็กกว่า NLTK. GPT-2 BPE แบ่ง token ในระดับ subword เช่น 'financial' → ['fin','anc','ial'] ทำให้รับมือคำหายากได้ดีกว่า Word-level.

📈 SUMMARY
   Raw articles      : 633
   After filtering   : 372
   Yahoo Finance                      : 48
   Motley Fool                        : 44
   CNBC - Top News                    : 30
   CNBC - Markets                     : 29
   CNBC - Earnings              

In [ ]:
# ══════════════════════════════════════════════════════════
#  SECTION 1.1 – DATA COLLECTION (RSS-based)
# ══════════════════════════════════════════════════════════

class RSSFinancialScraper:

    HEADERS = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept": "application/rss+xml, application/xml, text/xml, */*",
    }

    def __init__(self, target: int = 700, fetch_full_article: bool = True,
                 delay: tuple = (0.5, 1.5)):
        self.target = target
        self.fetch_full = fetch_full_article   # ดึงเนื้อหาเต็มจาก URL หรือใช้แค่ summary
        self.delay = delay
        self.articles: list[dict] = []
        self.seen_titles: set[str] = set()

    # ── Utilities ──────────────────────────────────────────

    def _sleep(self):
        time.sleep(random.uniform(*self.delay))

    def _get_html(self, url: str) -> BeautifulSoup | None:
        """ดึง HTML page สำหรับ full article."""
        try:
            self._sleep()
            r = requests.get(url, headers=self.HEADERS, timeout=12)
            r.raise_for_status()
            return BeautifulSoup(r.text, "html.parser")
        except Exception as e:
            logger.debug(f"[HTML ERROR] {url[:60]} → {e}")
            return None

    def _extract_full_content(self, url: str) -> str:
        """ดึงเนื้อหาเต็มจาก article URL."""
        soup = self._get_html(url)
        if not soup:
            return ""
        # ลอง selector ทั่วไป
        for selector in [
            {"tag": "article"},
            {"tag": "div", "class_": re.compile(r"article.body|story.body|content.body|post.body", re.I)},
            {"tag": "div", "id": re.compile(r"article|content|main", re.I)},
        ]:
            tag = selector.pop("tag")
            el = soup.find(tag, **selector)
            if el:
                paras = el.find_all("p")
                text = " ".join(p.get_text(" ", strip=True) for p in paras)
                if len(text.split()) > 30:
                    return text
        # fallback: ดึง <p> ทั้งหมด
        paras = soup.find_all("p")
        return " ".join(p.get_text(" ", strip=True) for p in paras)

    def _parse_rss(self, feed_url: str, source: str) -> int:
        """
        Parse RSS feed ด้วย feedparser และ requests
        คืนค่าจำนวน article ที่เพิ่มได้
        """
        added = 0
        try:
            # ใช้ feedparser (รองรับ Atom + RSS ทุก version)
            feed = feedparser.parse(feed_url, agent=self.HEADERS["User-Agent"])

            if feed.bozo and not feed.entries:
                # feedparser บางครั้ง bozo=True แต่ยังได้ entries
                logger.warning(f"[RSS WARN] {source}: {getattr(feed, 'bozo_exception', 'unknown')}")
                return 0

            for entry in feed.entries:
                if len(self.articles) >= self.target:
                    break

                # ── Title ──────────────────────────
                title = getattr(entry, "title", "").strip()
                if not title or title in self.seen_titles:
                    continue
                self.seen_titles.add(title)

                # ── URL ────────────────────────────
                url = getattr(entry, "link", "")

                # ── Content / Summary ──────────────
                # ลำดับความสำคัญ: content > summary > description
                content = ""
                if hasattr(entry, "content") and entry.content:
                    content = entry.content[0].get("value", "")
                if not content and hasattr(entry, "summary"):
                    content = entry.summary
                if not content and hasattr(entry, "description"):
                    content = entry.description

                # ลบ HTML ออกจาก summary ที่อาจมี tags
                content = BeautifulSoup(content, "html.parser").get_text(" ", strip=True)

                # ── Optionally fetch full article ──
                if self.fetch_full and url and len(content.split()) < 80:
                    full = self._extract_full_content(url)
                    if full:
                        content = full

                # ── Published date ─────────────────
                pub = ""
                if hasattr(entry, "published"):
                    pub = entry.published
                elif hasattr(entry, "updated"):
                    pub = entry.updated

                if not content or len(content.split()) < 10:
                    continue

                self.articles.append({
                    "id": len(self.articles) + 1,
                    "source": source,
                    "title": title,
                    "content": content,
                    "url": url,
                    "published": pub,
                    "scraped_at": datetime.now().isoformat(),
                })
                added += 1

        except Exception as e:
            logger.warning(f"[RSS ERROR] {source}: {e}")

        return added

    # ── Main Collection ────────────────────────────────────

    def collect(self) -> pd.DataFrame:
        logger.info(f"🚀 เริ่ม RSS Scraping เป้าหมาย {self.target} รายการ จาก {len(RSS_FEEDS)} feeds")

        for i, feed in enumerate(RSS_FEEDS):
            if len(self.articles) >= self.target:
                break
            source = feed["source"]
            url = feed["url"]
            logger.info(f"  [{i+1:>2}/{len(RSS_FEEDS)}] {source}...")
            n = self._parse_rss(url, source)
            logger.info(f"         +{n} รายการ | รวม {len(self.articles)}/{self.target}")

        logger.info(f"✅ รวบรวมได้ {len(self.articles)} รายการ")
        return pd.DataFrame(self.articles)

In [ ]:

# ══════════════════════════════════════════════════════════
#  SECTION 1.2 – PREPROCESSING PIPELINE
# ══════════════════════════════════════════════════════════

class PreprocessingPipeline:
    """
    1.2.1 ทำความสะอาด: HTML tags, URLs, Emojis, whitespace
    1.2.2 Tokenization Comparison: NLTK vs spaCy vs Subword BPE
    """

    def __init__(self):
        self._init_nltk()
        self._init_spacy()

    def _init_nltk(self):
        import nltk
        for pkg in ["punkt", "punkt_tab", "stopwords", "wordnet"]:
            nltk.download(pkg, quiet=True)
        from nltk.tokenize import word_tokenize
        from nltk.corpus import stopwords
        self.nltk_word_tokenize = word_tokenize
        self.stopwords_en = set(stopwords.words("english"))

    def _init_spacy(self):
        try:
            import spacy
            try:
                self.nlp = spacy.load("en_core_web_sm")
            except OSError:
                os.system("python -m spacy download en_core_web_sm")
                self.nlp = spacy.load("en_core_web_sm")
        except Exception as e:
            logger.warning(f"spaCy unavailable: {e}")
            self.nlp = None

    # ── 1.2.1 Cleaning ────────────────────────────────────

    @staticmethod
    def remove_html(text: str) -> str:
        return BeautifulSoup(text, "html.parser").get_text(separator=" ")

    @staticmethod
    def remove_urls(text: str) -> str:
        text = re.sub(r"http[s]?://\S+", " ", text)
        text = re.sub(r"www\.\S+", " ", text)
        return text

    @staticmethod
    def remove_emojis(text: str) -> str:
        emoji_re = re.compile(
            "[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF"
            "\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF"
            "\U00002702-\U000027B0\U000024C2-\U0001F251]+",
            flags=re.UNICODE,
        )
        return emoji_re.sub(" ", text)

    @staticmethod
    def normalize_whitespace(text: str) -> str:
        text = re.sub(r"[^a-zA-Z0-9\s.,!?;:'\"-]", " ", text)
        return re.sub(r"\s+", " ", text).strip()

    def clean(self, text: str) -> str:
        """Pipeline: HTML → URL → Emoji → normalize"""
        text = self.remove_html(text)
        text = self.remove_urls(text)
        text = self.remove_emojis(text)
        text = self.normalize_whitespace(text)
        return text

    # ── 1.2.2 Tokenizers ─────────────────────────────────

    def tok_nltk(self, text: str) -> list[str]:
        """Word-level: NLTK word_tokenize + stopword removal"""
        tokens = self.nltk_word_tokenize(text.lower())
        return [t for t in tokens if t.isalpha() and t not in self.stopwords_en]

    def tok_spacy(self, text: str) -> list[str]:
        """Word-level: spaCy lemmatize + stopword removal"""
        if not self.nlp:
            return []
        doc = self.nlp(text[:100_000])
        return [tok.lemma_.lower() for tok in doc
                if tok.is_alpha and not tok.is_stop and len(tok.text) > 1]

    def tok_subword(self, text: str) -> list[str]:
        """Subword BPE ด้วย GPT-2 tokenizer (transformers)"""
        try:
            from transformers import GPT2Tokenizer
            tok = GPT2Tokenizer.from_pretrained("gpt2")
            return tok.tokenize(text[:512])
        except ImportError:
            # fallback ถ้าไม่ได้ติดตั้ง transformers
            return re.sub(r"([a-z])([A-Z])", r"\1 \2", text).lower().split()

    # ── Comparison Report ─────────────────────────────────

    def compare(self, text: str) -> dict:
        """เปรียบเทียบ NLTK vs spaCy (Word-level) vs GPT-2 BPE (Subword-level)"""
        sample = text[:500]
        n_toks = self.tok_nltk(sample)
        s_toks = self.tok_spacy(sample)
        b_toks = self.tok_subword(sample)

        n_vocab = set(n_toks)
        s_vocab = set(s_toks)

        return {
            "sample_text": sample,
            "NLTK_word_level": {
                "method": "word_tokenize + stopword removal",
                "token_count": len(n_toks),
                "unique_vocab": len(n_vocab),
                "sample": n_toks[:15],
            },
            "spaCy_word_level": {
                "method": "spaCy lemmatize + stopword removal",
                "token_count": len(s_toks),
                "unique_vocab": len(s_vocab),
                "sample": s_toks[:15],
            },
            "GPT2_subword_BPE": {
                "method": "GPT-2 Byte-Pair Encoding (Subword-level)",
                "token_count": len(b_toks),
                "sample": b_toks[:15],
            },
            "analysis": {
                "overlap_nltk_spacy": len(n_vocab & s_vocab),
                "nltk_only": len(n_vocab - s_vocab),
                "spacy_only": len(s_vocab - n_vocab),
                "insight": (
                    "spaCy ใช้ lemmatization (running→run) ทำให้ vocab เล็กกว่า NLTK. "
                    "GPT-2 BPE แบ่ง token ในระดับ subword เช่น 'financial' → ['fin','anc','ial'] "
                    "ทำให้รับมือคำหายากได้ดีกว่า Word-level."
                ),
            },
        }

    # ── Run Pipeline ──────────────────────────────────────

    def run(self, df: pd.DataFrame) -> pd.DataFrame:
        logger.info("🧹 Preprocessing Pipeline...")

        # Step 1: clean
        logger.info("  [1/4] ทำความสะอาด text...")
        df["title_clean"]   = df["title"].fillna("").apply(self.clean)
        df["content_clean"] = df["content"].fillna("").apply(self.clean)
        df["word_count"]    = df["content_clean"].str.split().str.len()

        # Step 2: filter short articles
        before = len(df)
        df = df[df["word_count"] >= 20].reset_index(drop=True)
        logger.info(f"  [2/4] ลบ article สั้นเกินไป: {before - len(df)} rows ออก")

        # Step 3: NLTK tokenize
        logger.info("  [3/4] NLTK tokenization...")
        df["tokens_nltk"]  = df["content_clean"].apply(lambda x: self.tok_nltk(x[:3000]))
        df["count_nltk"]   = df["tokens_nltk"].apply(len)

        # Step 4: spaCy tokenize
        logger.info("  [4/4] spaCy tokenization...")
        df["tokens_spacy"] = df["content_clean"].apply(lambda x: self.tok_spacy(x[:3000]))
        df["count_spacy"]  = df["tokens_spacy"].apply(len)

        logger.info(f"✅ เสร็จ เหลือ {len(df)} รายการ")
        return df

In [ ]:
# ══════════════════════════════════════════════════════════
#  MAIN
# ══════════════════════════════════════════════════════════

def main():
    OUT = "output_data"
    os.makedirs(OUT, exist_ok=True)

    # ─── 1.1 Collect ──────────────────────────────────────
    scraper = RSSFinancialScraper(
        target=700,
        fetch_full_article=True,   # True = ดึงเนื้อหาเต็ม (ช้าลงนิด)
        delay=(0.5, 1.5),
    )
    raw_df = scraper.collect()
    raw_df.to_csv(f"{OUT}/raw_news.csv", index=False, encoding="utf-8-sig")
    logger.info(f"💾 raw_news.csv ({len(raw_df)} rows)")

    # ─── 1.2 Preprocess ───────────────────────────────────
    pipeline = PreprocessingPipeline()
    proc_df = pipeline.run(raw_df.copy())

    # serialize token lists เป็น JSON string ก่อน save
    proc_df["tokens_nltk"]  = proc_df["tokens_nltk"].apply(json.dumps)
    proc_df["tokens_spacy"] = proc_df["tokens_spacy"].apply(json.dumps)
    proc_df.to_csv(f"{OUT}/processed_news.csv", index=False, encoding="utf-8-sig")
    logger.info(f"💾 processed_news.csv ({len(proc_df)} rows)")

    # ─── Tokenization Comparison ──────────────────────────
    if not proc_df.empty:
        sample_text = raw_df["content"].iloc[0]
        report = pipeline.compare(sample_text)
        with open(f"{OUT}/tokenization_comparison.json", "w", encoding="utf-8") as f:
            json.dump(report, f, ensure_ascii=False, indent=2)

        print("\n" + "=" * 65)
        print("📊 TOKENIZATION COMPARISON")
        print("=" * 65)
        print(f"Sample: {report['sample_text'][:100]}...\n")
        for name, key in [("NLTK (Word-level)", "NLTK_word_level"),
                          ("spaCy (Word-level)", "spaCy_word_level"),
                          ("GPT-2 BPE (Subword)", "GPT2_subword_BPE")]:
            r = report[key]
            print(f"🔤 {name}")
            print(f"   Tokens : {r['token_count']}")
            print(f"   Sample : {r['sample'][:8]}")
            print()
        print(f"💡 {report['analysis']['insight']}")
        print("=" * 65)

    # ─── Summary ──────────────────────────────────────────
    print(f"\n📈 SUMMARY")
    print(f"   Raw articles      : {len(raw_df)}")
    print(f"   After filtering   : {len(proc_df)}")
    if "source" in proc_df.columns:
        src_counts = proc_df["source"].value_counts()
        for src, cnt in src_counts.items():
            print(f"   {src:<35}: {cnt}")
    print(f"   Output dir        : {os.path.abspath(OUT)}")


if __name__ == "__main__":
    main()